In [1]:
import re
from pyvi import ViTokenizer
import pandas as pd

# Hàm tiền xử lý

In [2]:
# Danh sách thay thế từ viết tắt
ACRONYMS = {
    # 1 ký tự
    "r": "rồi", "a": "anh", "e": "em", "j": "gì", "k": "không", "m": "mình", 
    "t": "tôi", "b": "bạn", "h": "giờ", "s": "sao",
    
    # 2 ký tự
    "ko": "không", "k0": "không", "bt": "bình thường", "vn": "việt nam", "vs": "và",
    "cx": "cũng được", "đc": "được", "dc": "được", "nh": "nhưng", "đg": "đường",
    "nc": "nước", "ms": "mới", "bh": "bao giờ", "km": "khuyến mãi", "ae": "anh em",
    "sg": "sài gòn", "hn": "hà nội", "vk": "vợ", "ck": "chồng", "nv": "nhân viên",
    "mn": "mọi người", "qc": "quảng cáo", "sp": "sản phẩm", "sd": "sử dụng", "wa": "quá",

    # 3 ký tự
    "ntn": "như thế nào", "lun": "luôn", "trc": "trước", "chx": "chưa", "nhg": "nhưng",
    "spt": "sản phẩm tốt", "nch": "nói chung", "kbh": "không bao giờ",

    # 4 ký tự
    "tnao": "thế nào", "hqua": "hôm qua", "toẹt": "tuyệt", "nhưg": "nhưng",
    "hnay": "hôm nay", "thik": "thích",

    # Dài hơn
    "lquan": "liên quan", "nchung": "nói chung", "k_thể": "không thể"
}

# Thay thế từ viết tắt
def replace_acronyms(text):
    for abbr, full in ACRONYMS.items():
        text = re.sub(rf'\b{abbr}\b', full, text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile(
        "[" +
        "\U0001F600-\U0001F64F" +
        "\U0001F300-\U0001F5FF" +
        "\U0001F680-\U0001F6FF" +
        "\U0001F1E0-\U0001F1FF" +
        "\U00002700-\U000027BF" +
        "\U0001F900-\U0001F9FF" +
        "\U0001FA70-\U0001FAFF" +
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub('', text)

def normalize_vietnamese(text):
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ',
        'òe': 'oè', 'óe': 'oé', 'ỏe': 'oẻ', 'õe': 'oẽ', 'ọe': 'oẹ',
        'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ', 'ụy': 'uỵ',
        'uả': 'ủa', 'ả': 'ả', 'ố': 'ố', 'u´': 'ố', 'ỗ': 'ỗ', 'ồ': 'ồ',
        'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ', 'ầ': 'ầ', 'ỏ': 'ỏ',
        'ề': 'ề', 'ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở',
        'ỉ': 'ỉ', 'ẻ': 'ẻ', 'àk': ' à ', 'aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ',
        'ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á'
    }
    for k, v in replace_list.items():
        text = text.replace(k, v)
    text = re.sub(r'([a-zA-Zà-ỹ])\1{2,}', r'\1', text)  # ví dụ: đẹpppp → đẹp
    return text

def remove_special_characters(text):
    # Xóa đường dẫn, mentions, hashtag
    text = re.sub(r"http\S+|www\S+|@\S+|#\S+", " ", text)
    # Xóa toàn bộ ký tự không phải chữ hoặc khoảng trắng
    text = re.sub(r"[^a-zA-ZÀ-ỹà-ỹ\s]", " ", text)
    return text

# Hàm tổng: chuẩn hóa + làm sạch + tách từ
def preprocess_text(text):
    text = text.lower()
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    text = remove_emoji(text)
    text = normalize_vietnamese(text)
    text = remove_special_characters(text)
    text = replace_acronyms(text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [3]:
df = pd.read_csv('coopmart_review.csv')
df

,title,publishedAtDate,originalLanguage,text,textTranslated,stars,comment
0,Co.opmart Bình Triệu,2025-04-24T17:45:42.757Z,vi,"Nhiều hàng hoá để lựa chọn, thường xuyên có ch...",NaN,5,"Nhiều hàng hoá để lựa chọn, thường xuyên có ch..."
1,Co.opmart Bình Triệu,2025-04-07T04:12:59.369Z,vi,Dịch vụ tốt,NaN,5,Dịch vụ tốt
2,Co.opmart Bình Triệu,2025-03-30T13:47:13.978Z,vi,"Hôm nay ngày 30-3 lúc tầm 19h56 -20h30 , tôi t...",NaN,1,"Hôm nay ngày 30-3 lúc tầm 19h56 -20h30 , tôi t..."
3,Co.opmart Bình Triệu,2025-03-24T14:19:32.053Z,vi,Nhân viên tính tiền thái độ khó chịu ra mặt,NaN,1,Nhân viên tính tiền thái độ khó chịu ra mặt
4,Co.opmart Bình Triệu,2025-03-02T14:42:40.200Z,vi,Thật tuyệt,NaN,5,Thật tuyệt
...,...,...,...,...,...,...,...
31974,Co.opmart Cao Lãnh,2021-08-15T08:44:18.773Z,vi,Tốt,NaN,4,Tốt
31975,Co.opmart Cao Lãnh,2019-04-08T08:23:04.140Z,vi,Tốt,NaN,4,Tốt
31976,Co.opmart Cao Lãnh,2019-04-13T11:10:38.372Z,vi,Tốt,NaN,4,Tốt
31977,Co.opmart Cao Lãnh,2019-09-28T12:38:48.364Z,vi,Tuyệt,NaN,4,Tuyệt


In [4]:
df['comment'] = df['comment'].apply(preprocess_text)

# Xóa những dòng có comment bị null

In [5]:
df = df.dropna(subset=['comment'])  # Xóa dòng có NaN ở cột comment
df = df[df['comment'].str.strip() != '']  # Xóa dòng chỉ chứa khoảng trắng


In [6]:
df

,title,publishedAtDate,originalLanguage,text,textTranslated,stars,comment
0,Co.opmart Bình Triệu,2025-04-24T17:45:42.757Z,vi,"Nhiều hàng hoá để lựa chọn, thường xuyên có ch...",NaN,5,nhiều hàng hoá để lựa chọn thường xuyên có chư...
1,Co.opmart Bình Triệu,2025-04-07T04:12:59.369Z,vi,Dịch vụ tốt,NaN,5,dịch vụ tốt
2,Co.opmart Bình Triệu,2025-03-30T13:47:13.978Z,vi,"Hôm nay ngày 30-3 lúc tầm 19h56 -20h30 , tôi t...",NaN,1,hôm nay ngày lúc tầm giờ giờ tôi tính tiền ở q...
3,Co.opmart Bình Triệu,2025-03-24T14:19:32.053Z,vi,Nhân viên tính tiền thái độ khó chịu ra mặt,NaN,1,nhân viên tính tiền thái độ khó chịu ra mặt
4,Co.opmart Bình Triệu,2025-03-02T14:42:40.200Z,vi,Thật tuyệt,NaN,5,thật tuyệt
...,...,...,...,...,...,...,...
31974,Co.opmart Cao Lãnh,2021-08-15T08:44:18.773Z,vi,Tốt,NaN,4,tốt
31975,Co.opmart Cao Lãnh,2019-04-08T08:23:04.140Z,vi,Tốt,NaN,4,tốt
31976,Co.opmart Cao Lãnh,2019-04-13T11:10:38.372Z,vi,Tốt,NaN,4,tốt
31977,Co.opmart Cao Lãnh,2019-09-28T12:38:48.364Z,vi,Tuyệt,NaN,4,tuyệt


# Tạo file đã tiền xử lý, lấy 2 cột là stars và comment

In [7]:
df2 = df[['comment',"stars"]]

In [10]:
df2.to_csv('coopmart_review_preprocess.csv', encoding='utf-8-sig', index=False)